In [ ]:
import pandas as pd
import glob
import re
import numpy as np

In [ ]:
fList=glob.glob('logs/DAQ_logs/lc-ASIC*csv')

In [ ]:
x=re.compile('logs/DAQ_logs/lc-ASIC_*(.*)_(\d\d)mar_(\d*).csv')

In [ ]:
timestamp=[]
extraInfo=[]
errorInfo=[]
for f in fList[:]:
    extra_,date_,time_=x.match(f).groups()
    dfASIC=pd.read_csv(f)
    dfEm=pd.read_csv(f.replace('ASIC','emulator'))
    try:
        bits_ASIC=np.array([[*f"{int(''.join(x),16):0256b}"] for x in dfASIC.values[:,:8]],dtype=int)
        bits_Em=np.array([[*f"{int(''.join(x),16):0256b}"] for x in dfEm.values[:,:8]],dtype=int)
        bitErrors=~(bits_ASIC==bits_Em)
        errorInfo.append([bitErrors.any(axis=1).sum(),bitErrors[:30].sum(),bitErrors[30:330].sum(),bitErrors[330:1000].sum(),bitErrors[1000:2000].sum(),bitErrors[2000:].sum()])
        timestamp.append(f'03-{date_}-22 {time_[:2]}:{time_[2:4]}:{time_[4:]}')
        extraInfo.append(extra_)
    except:
        print('ERROR WITH',f)

In [ ]:
df=pd.DataFrame(errorInfo,index=pd.to_datetime(timestamp),columns=['BX_Err_Cnt','Bit_Err_Early','Bit_Err_Spill','Bit_Err_ShortAfter','Bit_Err_MidAfter','Bit_Err_LongAfter'])

In [ ]:
df['ExtraInfo']=extraInfo

In [ ]:
df

In [ ]:
df.to_csv('logs/BitErrorCounts.csv')